In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from fbprophet import Prophet

In [2]:
df = pd.read_csv("/Users/shakilafarjana/Downloads/country_vaccinations.csv")

In [3]:
df.head(2)

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...


In [4]:
df.shape

(15666, 15)

In [5]:
df1 = df[["country", "total_vaccinations", "people_fully_vaccinated", "vaccines"]]
df1.head(3)

,country,total_vaccinations,people_fully_vaccinated,vaccines
0,Afghanistan,0.0,NaN,Oxford/AstraZeneca
1,Afghanistan,NaN,NaN,Oxford/AstraZeneca
2,Afghanistan,NaN,NaN,Oxford/AstraZeneca


In [6]:
df1["country"].unique().shape

(196,)

In [7]:
df1["vaccines"].unique().shape

(34,)

In [8]:
df1.shape

(15666, 4)

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15666 entries, 0 to 15665
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  15666 non-null  object 
 1   total_vaccinations       9437 non-null   float64
 2   people_fully_vaccinated  6502 non-null   float64
 3   vaccines                 15666 non-null  object 
dtypes: float64(2), object(2)
memory usage: 489.7+ KB


In [10]:
country_vaccine = df1['country'] +' '+ df1['vaccines']
country_vaccine.nunique()

196

In [11]:
unique_vaccine = df1['vaccines'].unique().tolist()
sorted(unique_vaccine)[:20]

['CanSino, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V',
 'CanSino, Sinopharm/Beijing, Sinovac, Sputnik V',
 'Covaxin, Oxford/AstraZeneca',
 'EpiVacCorona, Sputnik V',
 'Johnson&Johnson',
 'Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',
 'Johnson&Johnson, Moderna, Pfizer/BioNTech',
 'Moderna, Oxford/AstraZeneca',
 'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',
 'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',
 'Moderna, Pfizer/BioNTech',
 'Oxford/AstraZeneca',
 'Oxford/AstraZeneca, Pfizer/BioNTech',
 'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing',
 'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sinopharm/Wuhan, Sputnik V',
 'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',
 'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac',
 'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V',
 'Oxford/AstraZeneca, Pfizer/BioNTech, Sputnik V',
 'Oxford/AstraZeneca, Sinopharm/Beijing']

# *FuzzyWuzzy*
## *FuzzyWuzzy has four scorer options to find the Levenshtein distance between two strings. In this example, I would check on the token sort ratio and the token set ratio, for I believe they are more suitable for this dataset which might have mixed words order and duplicated words.*
## *I would pick four brand names and find their similar names in the Brand column. Since we're matching the Brand column with itself, the result would always include the selected name with a score of 100.*

# Token Sort Ratio
## *The token sort ratio scorer tokenizes the strings and cleans them by returning these strings to lower cases, removing punctuations, and then sorting them alphabetically. After that, it finds the Levenshtein distance and returns the similarity percentage.*

In [13]:
from fuzzywuzzy import process, fuzz

In [14]:
process.extract('7 Select', unique_vaccine, scorer=fuzz.token_sort_ratio)

[('Oxford/AstraZeneca', 31),
 ('Oxford/AstraZeneca, Pfizer/BioNTech', 24),
 ('Moderna, Oxford/AstraZeneca', 24),
 ('Covaxin, Oxford/AstraZeneca', 24),
 ('Moderna, Oxford/AstraZeneca, Pfizer/BioNTech', 20)]

In [15]:
process.extract('A-Sha', unique_vaccine, scorer=fuzz.token_sort_ratio)

[('Sinopharm/Beijing', 36),
 ('Oxford/AstraZeneca, Sinovac', 26),
 ('Sinopharm/Beijing, Sputnik V', 25),
 ('Oxford/AstraZeneca, Sinopharm/Beijing', 24),
 ('EpiVacCorona, Sputnik V', 22)]

In [16]:
score_sort = [(x,) + i
             for x in unique_vaccine 
             for i in process.extract(x, unique_vaccine, scorer=fuzz.token_sort_ratio)]


In [39]:
#Create dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])
similarity_sort.head(3)

,brand_sort,match_sort,score_sort
0,Oxford/AstraZeneca,Oxford/AstraZeneca,100
1,Oxford/AstraZeneca,"Oxford/AstraZeneca, Sinovac",82
2,Oxford/AstraZeneca,"Moderna, Oxford/AstraZeneca",82


In [40]:
#Derive representative values
similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['brand_sort'], similarity_sort['match_sort'])
similarity_sort.head(2)

,brand_sort,match_sort,score_sort,sorted_brand_sort
0,Oxford/AstraZeneca,Oxford/AstraZeneca,100,Oxford/AstraZeneca
1,Oxford/AstraZeneca,"Oxford/AstraZeneca, Sinovac",82,Oxford/AstraZeneca


In [19]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                                      (similarity_sort['brand_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'])]

In [20]:
#Drop the representative value column
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()

In [21]:
#Group matches by brand names and scores
#pd.set_option('display.max_rows', None)
high_score_sort.groupby(['brand_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
brand_sort,score_sort,
"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V",94,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."
"CanSino, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V",93,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ..."
"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V",92,"Oxford/AstraZeneca, Pfizer/BioNTech, Sputnik V"
"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing",91,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."
"Oxford/AstraZeneca, Sinopharm/Beijing",90,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac"
"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac",89,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ..."
"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sinopharm/Wuhan, Sputnik V",89,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."
"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac",89,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V"
"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech",89,"Oxford/AstraZeneca, Pfizer/BioNTech"


In [28]:
#Souper - Super - 91%
df1[(df1['vaccines'] == 'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech') | (df1['vaccines'] == 'Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech')].sort_values(['vaccines'])

,country,total_vaccinations,people_fully_vaccinated,vaccines
746,Austria,732.0,NaN,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ..."
7260,Italy,22143128.0,6700605.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ..."
7259,Italy,21673864.0,6548151.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ..."
7258,Italy,21232972.0,6404916.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ..."
7257,Italy,20825126.0,6273699.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ..."
...,...,...,...,...
3649,Denmark,51277.0,NaN,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
3648,Denmark,45435.0,NaN,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
3647,Denmark,36745.0,NaN,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
3645,Denmark,28412.0,NaN,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"


In [27]:
df1[(df1['vaccines'] == 'Oxford/AstraZeneca, Sinovac') | (df1['vaccines'] == 'Covaxin, Oxford/AstraZeneca')].sort_values(['vaccines'])

,country,total_vaccinations,people_fully_vaccinated,vaccines
6464,India,58109773.0,8683155.0,"Covaxin, Oxford/AstraZeneca"
6426,India,8857341.0,216339.0,"Covaxin, Oxford/AstraZeneca"
6427,India,9186757.0,342116.0,"Covaxin, Oxford/AstraZeneca"
6428,India,9846523.0,464932.0,"Covaxin, Oxford/AstraZeneca"
6429,India,10449942.0,756942.0,"Covaxin, Oxford/AstraZeneca"
...,...,...,...,...
2061,Brazil,8101787.0,1755018.0,"Oxford/AstraZeneca, Sinovac"
2060,Brazil,7799000.0,1596945.0,"Oxford/AstraZeneca, Sinovac"
2059,Brazil,7551676.0,1435594.0,"Oxford/AstraZeneca, Sinovac"
2071,Brazil,NaN,NaN,"Oxford/AstraZeneca, Sinovac"


# Token Set Ratio
## We will go over the same steps as above.

In [29]:
#Create tuples of brand names, matched brand names, and the score
score_set = [(x,) + i
             for x in unique_vaccine 
             for i in process.extract(x, unique_vaccine, scorer=fuzz.token_set_ratio)]

In [30]:
#Create dataframe from the tuples and derive representative values
similarity_set = pd.DataFrame(score_set, columns=['brand_set','match_set','score_set'])
similarity_set['sorted_brand_set'] = np.minimum(similarity_set['brand_set'], similarity_set['match_set'])

#Pick values
high_score_set = similarity_set[(similarity_set['score_set'] >= 80) & 
                                    (similarity_set['brand_set'] != similarity_set['match_set']) & 
                                    (similarity_set['sorted_brand_set'] != similarity_set['match_set'])]

#Drop the representative value column
high_score_set = high_score_set.drop('sorted_brand_set',axis=1).copy()

## Since the token set ratio scorer will tolerate more 'noise' when matching two values. I would group the result by matched values to reduce the number of rows.

In [31]:
#Group brands by matches and scores
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
high_score_set.groupby(['match_set','score_set']).agg(
                       {'brand_set': ', '.join}).sort_values(
                       ['score_set'], ascending=False)

,,brand_set
match_set,score_set,
"Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech",100,Johnson&Johnson
"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V",100,"Oxford/AstraZeneca, Oxford/AstraZeneca, Pfizer/BioNTech, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, CanSino, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V"
"Sinopharm/Beijing, Sputnik V",100,"CanSino, Sinopharm/Beijing, Sinovac, Sputnik V"
"Sinopharm/Beijing, Sinopharm/Wuhan, Sinovac",100,Sinopharm/Beijing
Sinopharm/Beijing,100,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V, Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac, Oxford/AstraZeneca, Sinopharm/Beijing, Pfizer/BioNTech, Sinopharm/Beijing, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, CanSino, Sinopharm/Beijing, Sinovac, Sputnik V"
Pfizer/BioNTech,100,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V, Oxford/AstraZeneca, Pfizer/BioNTech, Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Moderna, Pfizer/BioNTech, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, CanSino, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V, Oxford/AstraZeneca, Pfizer/BioNTech, Sputnik V, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sinopharm/Wuhan, Sputnik V, Johnson&Johnson, Moderna, Pfizer/BioNTech"
"Oxford/AstraZeneca, Sputnik V",100,"Oxford/AstraZeneca, Sinovac, Sputnik V"
"Oxford/AstraZeneca, Sinovac",100,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac"
"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",100,"Oxford/AstraZeneca, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V, Oxford/AstraZeneca, Sinopharm/Beijing, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sinopharm/Wuhan, Sputnik V"


In [32]:
#Create columns with brand names combining scores
high_score_sort['brand_sort'] = high_score_sort['brand_sort'] + ': ' + high_score_sort['score_sort'].astype(str)
high_score_set['brand_set'] = high_score_set['brand_set'] + ': ' + high_score_set['score_set'].astype(str)

In [33]:
#Group data by matched name and store in new dataframe
token_sort = high_score_sort.groupby(['match_sort']).agg({'brand_sort': ', '.join}).reset_index()
token_set = high_score_set.groupby(['match_set']).agg({'brand_set': ', '.join}).reset_index()

#Rename columns
token_sort = token_sort.rename(columns={'match_sort':'brand'})
token_set = token_set.rename(columns={'match_set':'brand'})


In [34]:
#Outer join two tables by brand (matched names)
similarity = pd.merge(token_sort, token_set, how='outer', on='brand')

#Replace NaN values and rename columns for readability
similarity = similarity.replace(np.nan,'')
similarity = similarity.rename(columns={'brand_set':'token_set_ratio','brand_sort':'token_sort_ratio'})

In [37]:
similarity.sort_values('brand').reset_index(drop=True)

,brand,token_sort_ratio,token_set_ratio
0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech",,Johnson&Johnson: 100
1,"Johnson&Johnson, Moderna, Pfizer/BioNTech","Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech: 80",Johnson&Johnson: 100
2,"Moderna, Oxford/AstraZeneca","Covaxin, Oxford/AstraZeneca: 81",
3,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech","Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech: 84","Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech: 100, Moderna, Oxford/AstraZeneca: 100"
4,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V",,"Moderna, Oxford/AstraZeneca: 100"
5,"Moderna, Pfizer/BioNTech",,"Johnson&Johnson, Moderna, Pfizer/BioNTech: 100"
6,Oxford/AstraZeneca,"Moderna, Oxford/AstraZeneca: 82, Covaxin, Oxford/AstraZeneca: 82","Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech: 100, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech: 100, Moderna, Oxford/AstraZeneca: 100, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V: 100, Covaxin, Oxford/AstraZeneca: 100, CanSino, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V: 100"
7,"Oxford/AstraZeneca, Pfizer/BioNTech","Moderna, Oxford/AstraZeneca, Pfizer/BioNTech: 89","Oxford/AstraZeneca: 100, Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech: 100, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech: 100, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V: 100, Covaxin, Oxford/AstraZeneca: 82, CanSino, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V: 100"
8,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing","Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V: 85",
9,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sinopharm/Wuhan, Sputnik V","Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V: 84",


### Now we can see how different it is between two scorers. As expected, the token set ratio matches wrong names with high scores (e.g. S&S, Mr.Noodles). However, it does bring in more matches that the token sort ratio could not get (e.g. 7 Select/Nissin, Sugakiya Foods, Vina Acecook).
### It would be beneficial to apply both methods in this case.

In [24]:
df2[["country", "vaccines"]].value_counts()

country        vaccines                                    
Canada         Moderna, Oxford/AstraZeneca, Pfizer/BioNTech    144
China          Sinopharm/Beijing, Sinopharm/Wuhan, Sinovac     143
Russia         EpiVacCorona, Sputnik V                         143
Israel         Moderna, Pfizer/BioNTech                        139
United States  Johnson&Johnson, Moderna, Pfizer/BioNTech       138
                                                              ... 
Congo          Sinopharm/Beijing, Sputnik V                      1
Timor          Oxford/AstraZeneca                                1
Armenia        Sputnik V                                         1
Somalia        Oxford/AstraZeneca, Sinopharm/Beijing             1
Libya          Oxford/AstraZeneca, Sinovac, Sputnik V            1
Length: 196, dtype: int64

In [10]:
df2.shape

(15666, 4)

In [69]:
df1.groupby(["country", "total_vaccinations"]).size().groupby(level=1).max()

total_vaccinations
0.0            120
1.0              1
2.0              2
5.0              1
6.0              5
              ... 
275338000.0      1
279905000.0      1
284595000.0      1
289627000.0      1
297734000.0      1
Length: 9099, dtype: int64

In [83]:
df1.groupby(["country", "total_vaccinations"]).size()

country      total_vaccinations
Afghanistan  0.0                   56
             8200.0                 1
             54000.0                1
             120000.0               1
             240000.0               1
                                   ..
Zimbabwe     524199.0               1
             537516.0               1
             559777.0               1
             576233.0               1
             607355.0               1
Length: 9365, dtype: int64

In [84]:
df1.shape

(15666, 4)

In [72]:
df1.sort_values(['country', 'total_vaccinations'], ascending=[True, False], inplace=True)

In [73]:
df1.head()

,country,total_vaccinations,total_vaccinations_per_hundred,vaccines
59,Afghanistan,240000.0,0.62,Oxford/AstraZeneca
44,Afghanistan,120000.0,0.31,Oxford/AstraZeneca
22,Afghanistan,54000.0,0.14,Oxford/AstraZeneca
6,Afghanistan,8200.0,0.02,Oxford/AstraZeneca
0,Afghanistan,0.0,0.00,Oxford/AstraZeneca


In [74]:
df1.groupby("country").count().sort_values(by = "total_vaccinations")

,total_vaccinations,total_vaccinations_per_hundred,vaccines
country,,,
Djibouti,1,1,1
Congo,1,1,1
Comoros,1,1,1
Armenia,1,1,1
Somalia,1,1,1
...,...,...,...
United States,138,138,138
Israel,139,139,139
China,143,143,143


In [34]:
df2 = df1.groupby("country").count()

In [77]:
df2.head()

,country,No_of_country
0,Afghanistan,1
1,Albania,1
2,Algeria,1
3,Andorra,1
4,Angola,1


In [78]:
df['total_vaccinations'].argmax()

2945

In [80]:
df.sort_values(by=['total_vaccinations'])

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,total_ccountry,total_country
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...,60,60
4715,Fiji,FJI,2021-03-17,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Oxford/AstraZeneca,Ministry of Health,https://twitter.com/FijiPM/status/138294891596...,31,31
5002,Gabon,GAB,2021-03-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Sinopharm/Beijing,Ministry of Health,https://twitter.com/SanteGOUVGA/status/1385685...,33,33
5035,Gambia,GMB,2021-03-09,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Oxford/AstraZeneca,Ministry of Health,https://twitter.com/MoHCovid19GMB/status/13860...,47,47
5082,Georgia,GEO,2021-03-14,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Oxford/AstraZeneca,Government of Georgia,https://www.facebook.com/ncdcgeorgia/photos/a....,54,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15578,Zambia,ZMB,2021-04-27,NaN,NaN,NaN,NaN,2270.0,NaN,NaN,NaN,123.0,Oxford/AstraZeneca,Government of Zambia,https://www.facebook.com/mohzambia/photos/a.77...,23,23
15582,Zambia,ZMB,2021-05-01,NaN,NaN,NaN,NaN,3438.0,NaN,NaN,NaN,187.0,Oxford/AstraZeneca,Government of Zambia,https://www.facebook.com/mohzambia/photos/a.77...,23,23
15589,Zimbabwe,ZWE,2021-02-19,NaN,NaN,NaN,NaN,328.0,NaN,NaN,NaN,22.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1390390738...,78,78
15590,Zimbabwe,ZWE,2021-02-20,NaN,NaN,NaN,NaN,328.0,NaN,NaN,NaN,22.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1390390738...,78,78
